# Load Data

In [109]:
# Imports
# from settings import *
# import analyze_cascade
import datetime
import pandas as pd
import numpy as np
from itertools import groupby
import random
import math
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [2]:
metadata_file = 'metadata_anon.txt'

In [3]:
# Read meta data 
fin = open(metadata_file,'r')
lines = fin.readlines()
fin.close()
cascade_id2metadata={}
for line in lines:
    line = line.replace('\n','')
    item = eval(line)
    cascade_id2metadata[item[0]] = item[1]

## Descriptives of dynamic measures

In [ ]:
len_depth2time = []
len_num_followees_list = []
len_depth2uu = []
len_uu2time = []
len_depth2breadth = []
for cascade,metadata in cascade_id2metadata.items():
    len_depth2time.append(len(metadata['depth2time'].keys()))
    len_num_followees_list.append(len(metadata['num_followees_list']))
    len_depth2uu.append(len(metadata['depth2uu'].keys()))
    len_uu2time.append(len(metadata['uu2time'].keys()))
    len_depth2breadth.append(len(metadata['depth2breadth'].keys()))
    
# Convert to data frame
df_len = pd.DataFrame({'depth2time ': len_depth2time, 'num_followees_list': len_num_followees_list, 'depth2uu': len_depth2uu, 
                       'uu2time': len_uu2time, 'depth2breadth': len_depth2breadth})

# # Get summary
df_len.describe(percentiles = [0.25, 0.5, 0.75, 0.8, 0.85, 0.9, 0.95])
    

## Dynamic Measure data set


In [194]:
# Function to get expression of each item in a dictionary entry
def get_expression_list(entry):
    expression = []
    for i in entry.keys():
        expression.append(float(entry[i]))
    return expression

# Convert y to classification
def veracity_to_categorical(v):
    if v == 'FALSE':
        vbin = [1,0,0]
    elif v == 'MIXED':
        vbin = [0,1,0]
    elif v == 'TRUE':
        vbin = [0,0,1]
    return vbin

# Get data in list format
data = []
for cascade,metadata in cascade_id2metadata.items():
    if metadata['virality'] is not None:       
        # Get depth
        depth2time = get_expression_list(metadata['depth2time'])
        depth2uu = get_expression_list(metadata['depth2uu'])
        depth2breadth = get_expression_list(metadata['depth2breadth']) 
        veracity = veracity_to_categorical(metadata['veracity'])
        data_id = []
        for time, uu, breadth in zip(depth2time, depth2uu, depth2breadth):
            data_t = [cascade, veracity, time, uu, breadth]
            data_id.append(data_t)
        data.extend([data_id])
        
# Function: Create training and test set
def split_list(lst, train_size): # train_size is a proportion
    split = len(lst) * train_size
    if split.is_integer():
        split = int(split)
        return lst[:split], lst[split:]
    else:
        split = math.floor(split) + 1
        return lst[:split], lst[split:]
    
# Function: Padding for groups of equal batches
def padding(lst, bsize):
    if len(lst) % bsize != 0:
        psize = bsize - (len(lst) % 5)
        samples = random.choices(lst, k=psize)
        lst.extend(samples)
    return lst

# # # Separate id, x and y
def separate(list_in_list):
    cid = []
    y = []
    for lst in list_in_list:
        cid.append(lst[0][0]) # only one id is needed
#         veracity_id = []
#         for sublist in lst:
#             veracity_id.extend([sublist[1]])
#         veracity.append(veracity_id)
        y.append(lst[0][1])
    x = []
    for lst in list_in_list:
        x_id = []
        for sublist in lst:
            x_id.append(sublist[2:])
        x.extend([x_id])
    return cid, y, x

# # Group by sequence length and append to have batches of 5 for both training and test
data.sort(key=len)   # Randomly reshuffle before? random.shuffle(data_train_padded)
x_train = []
x_test = []
y_train = []
y_test = []
cid_train = []
cid_test = []
for k, g in groupby(data, len):
    group = list(g)
    if len(group) > 2: # This omits too small groups
        # Create train and test bucket
        split_list(group, 0.5)
        train_group, test_group = split_list(group, 0.5)
        # Padd for equal batch size
        train_group_padded = padding(train_group, 5)
        test_group_padded = padding(test_group, 5)
        # Separate list
        cid_train_group, y_train_group, x_train_group = separate(train_group_padded)
        cid_test_group, y_test_group, x_test_group = separate(test_group_padded)
        # Append:  convert y and x into numpy array
        x_train.append(np.array(x_train_group))
        x_test.append(np.array(x_test_group))
        y_train.append(np.array(y_train_group))
        y_test.append(np.array(y_test_group))
        cid_train.append(cid_train_group)
        cid_test.append(cid_test_group)
        
# # Function to standardize the list
# def standardization(lst, index, mean, std):
#     for array3d in lst:
#         for array2d in array3d:
#             for vector in array2d:
#                 vector[index] = (vector[index] - mean) / std
#     return lst

# # Function to compute mean and std of variable and then standardizes this variable in list
# def standardize_data(a_list, b_list, index):
#     var = []
#     # Compute mean and std from train data variable
#     for array3d in a_list:
#         for array2d in array3d:
#             for vector in array2d:
#                 var.append(vector[index])
#     var = np.array(var)
#     var_mean = var.mean()
#     var_std = var.std()
#     # Standardize a
#     a_list_std = standardization(a_list, index, var_mean, var_std)
#     b_list_std = standardization(b_list, index, var_mean, var_std)
#     return a_list_std, b_list_std

# # Standardize all variables
# def standardize_all(a_list, b_list):
#     length = len(a_list[0][0][0])
#     indices = list(range(length))
#     for i in indices:
#         std_a, std_b = standardize_data(a_list, b_list, i)
#     return std_a, std_b

# x_train, x_test = standardize_all(x_train, x_test)

## LSTM on dynamic measures

In [196]:
model = Sequential()
model.add(LSTM(5, input_shape = (None, 3),  return_sequences = False))
model.add(Dense(3, activation='softmax'))
# model = M.Model(inputs=model_input, outputs=model_output)
# model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
for X,Y in zip(x_train, y_train):
    hist = model.fit(X, Y, epochs=5, batch_size=5)
    
# model.fit(std_depth_train, cat_veracity_train, epochs=100, shuffle=False)


Epoch 1/10
12960/12960 [==============================] - 12s 917us/step - loss: 0.5275 - acc: 0.7897
Epoch 2/10
12960/12960 [==============================] - 10s 788us/step - loss: 0.4584 - acc: 0.8108
Epoch 3/10
12960/12960 [==============================] - 10s 789us/step - loss: 0.4499 - acc: 0.8368
Epoch 4/10
12960/12960 [==============================] - 10s 778us/step - loss: 0.4456 - acc: 0.8424
Epoch 5/10
12960/12960 [==============================] - 10s 796us/step - loss: 0.4429 - acc: 0.8424
Epoch 6/10
12960/12960 [==============================] - 10s 792us/step - loss: 0.4409 - acc: 0.8425
Epoch 7/10
12960/12960 [==============================] - 11s 832us/step - loss: 0.4400 - acc: 0.8425
Epoch 8/10
12960/12960 [==============================] - 11s 842us/step - loss: 0.4393 - acc: 0.8425
Epoch 9/10
12960/12960 [==============================] - 11s 845us/step - loss: 0.4386 - acc: 0.8425
Epoch 10/10
12960/12960 [==============================] - 11s 844us/step - loss: 

Epoch 5/10
5/5 [==============================] - 0s 6ms/step - loss: 0.4634 - acc: 0.8000
Epoch 6/10
5/5 [==============================] - 0s 5ms/step - loss: 0.4631 - acc: 0.8000
Epoch 7/10
5/5 [==============================] - 0s 6ms/step - loss: 0.4627 - acc: 0.8000
Epoch 8/10
5/5 [==============================] - 0s 7ms/step - loss: 0.4622 - acc: 0.8000
Epoch 9/10
5/5 [==============================] - 0s 7ms/step - loss: 0.4617 - acc: 0.8000
Epoch 10/10
5/5 [==============================] - 0s 7ms/step - loss: 0.4610 - acc: 0.8000


## Data frame of static measures

In [ ]:
# Get static measures
veracity = []
virality = []
depth = []
breadth = []
size = []
verified = []
nfollowers = []
nfollowees = []
engagement = []
category = []
day = []
hour = []
for cascade,metadata in cascade_id2metadata.items():
    veracity.append(metadata['veracity'])
    virality.append(metadata['virality'])
    depth.append(metadata['depth'])
    breadth.append(metadata['max_breadth'])
    size.append(metadata['size'])
    verified.append(metadata['verified_list'][0])
    nfollowers.append(metadata['num_followers_list'][0])
    nfollowees.append(metadata['num_followees_list'][0])
    engagement.append(metadata['engagement_list'][0])
    category.append(metadata['rumor_category'])
    day.append(metadata['start_date'].day)
    hour.append(metadata['start_date'].hour)

# Convert to data frame
df = pd.DataFrame({'veracity': veracity, 'virality': virality, 'depth': depth, 'breadth': breadth, 'size': size, 'verified': verified, 'nfollowers': nfollowers, 
                   'nfollowees': nfollowees, 'engangement': engagement, 'category': category, 'day': day, 'hour': hour})

# Inspect
df.head(5)
